### Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
from docx import Document

### Считываем исходную таблицу из файла

In [2]:
def parse_docx_table(filename: str):
    document = Document(filename)

    tables = []
    for table in document.tables:
        # список списков с пустыми строками - структура будущего датафрейма
        df = [['' for x in range(len(table.columns))] for y in range(len(table.rows))]
        # заполняем дф данными из табл
        for x, row in enumerate(table.rows):
            for y, cell in enumerate(row.cells):
                if cell.text:
                    df[x][y] = cell.text
        tables.append(pd.DataFrame(df))

    return tables

In [3]:
# берем первую таблицу из списка
df = parse_docx_table('DE_task_table.docx')[0]
# заголовк таблицы - первый ряд
header = df.iloc[0]
# убираем первый ряд из таблицы
df = df[1:]
# назначаем название колонок дф
df.columns = header

In [10]:
# переводим формат дат, год 9999 заменяем на nan обработкой ошибок
df['start_date']= pd.to_datetime(df['start_date'], format="%d.%m.%Y", errors = 'coerce',)
df['finish_date']= pd.to_datetime(df['finish_date'], format="%d.%m.%Y", errors = 'coerce',)

In [11]:
# правило №2
df.loc[df['finish_date'].isnull(), 'finish_date'] = df['end_da']
df['start_date']= pd.to_datetime(df['start_date'], infer_datetime_format=True,)
df['finish_date']= pd.to_datetime(df['finish_date'], infer_datetime_format=True)

In [12]:
df

,tab_num,start_date,finish_date,wday_type01,wday_type02,wday_type03,wday_type04,wday_type05,wplace_type,end_da
1,15123,2020-09-02,2020-10-31,0,0,0,0,0,0,31.10.2020
2,16234,2020-09-20,2020-10-30,0,0,1,1,0,2,
3,17345,2020-09-28,2020-10-25,1,0,0,0,0,2,
4,17345,2020-10-26,2020-12-31,1,1,1,1,1,1,
5,18456,2020-09-02,2020-09-30,2,2,2,2,2,3,30.09.2020
6,19567,2020-09-02,2020-12-31,3,3,3,3,3,4,


In [13]:
# словарь для каждого номера ряда исходной таблицы с датой старта и окончания режима
start_date_dict = dict(zip(range(df.shape[0]), df.start_date.to_list()))
finish_date_dict = dict(zip(range(df.shape[0]), df.finish_date.to_list()))

In [18]:
dash = pd.DataFrame()
# для каждого ряда - свой датафрейм по правилам №1 и №2
for num, tab_num in enumerate(df.tab_num.to_list()):
    interim = pd.DataFrame()
    interim['ymd_date'] = pd.date_range(start='2020-09-01', end='2020-12-31', freq='D')
    # дополнительные фичи для правила №8
    interim['weekday'] = interim['ymd_date'].dt.dayofweek
    interim['week'] = interim['ymd_date'].dt.isocalendar().week
    interim['week'] = interim['week'] - interim['week'].min()
    interim['halfweek'] = interim['week'] // 2
    interim['tab_num'] = tab_num
    interim['start_date'] = start_date_dict[num]
    interim['finish_date'] = finish_date_dict[num]
    dash = pd.concat([dash, interim])
dash['to_be_at_office'] = np.nan

In [19]:
repeated_tab_num = [x for x in df[df['tab_num'].duplicated()].tab_num.values]
repeated_tab_num

['17345']

In [38]:
def process_duplicated_tab_num(init_df: pd.DataFrame, repeated_tab_nums: list):
    dash_merged = init_df.copy()
    for tab_num in repeated_tab_nums:
        dash_merged = pd.concat(
            [
                dash_merged[~dash_merged.tab_num.isin([tab_num])],
                pd.merge(
                    dash_merged.loc[(dash_merged['tab_num'] == tab_num)][['ymd_date', 'weekday', 'tab_num']],
                    dash_merged.loc[
                        (dash_merged['tab_num'] == tab_num) &
                        (dash_merged['ymd_date'] <= dash_merged['finish_date']) &
                        (dash_merged['ymd_date'] >= dash_merged['start_date'])
                    ],
                    how='left'
                ).drop_duplicates()
            ]
        )
        print(dash_merged.shape)
    return dash_merged

In [39]:
dash_merged = process_duplicated_tab_num(dash, repeated_tab_num)

(610, 8)


In [40]:
dash.shape, dash_merged.shape, 122*5

610

In [48]:
assert dash_merged.shape[0] == 122 * len(dash.tab_num.unique())

In [50]:
dash_merged.sample(5)

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,NaN
26,2020-09-27,6,3,1,16234,2020-09-20,2020-10-30,NaN
82,2020-11-22,6,11,5,16234,2020-09-20,2020-10-30,NaN
107,2020-12-17,3,15,7,19567,2020-09-02,2020-12-31,NaN
32,2020-10-03,5,4,2,19567,2020-09-02,2020-12-31,NaN


In [51]:
df.tab_num.unique()[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 121
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ymd_date         610 non-null    datetime64[ns]
 1   weekday          610 non-null    int64         
 2   week             583 non-null    UInt32        
 3   halfweek         583 non-null    UInt32        
 4   tab_num          610 non-null    object        
 5   start_date       583 non-null    datetime64[ns]
 6   finish_date      583 non-null    datetime64[ns]
 7   to_be_at_office  0 non-null      float64       
dtypes: UInt32(2), datetime64[ns](3), float64(1), int64(1), object(1)
memory usage: 39.3+ KB


In [52]:
print(df.tab_num.to_list()[0])
tab_num_index = 0
dash_merged.loc[
    (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 1

'15123'

In [35]:
dash_merged[dash_merged['tab_num'] == '15123']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,15123,2020-09-02,2020-10-31,NaN
118,2020-12-28,0,17,8,15123,2020-09-02,2020-10-31,NaN
119,2020-12-29,1,17,8,15123,2020-09-02,2020-10-31,NaN
120,2020-12-30,2,17,8,15123,2020-09-02,2020-10-31,NaN


In [36]:
print(df.tab_num.to_list()[3])

17345


In [37]:
tab_num_index = 3
dash_merged.loc[
    (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 0

In [38]:
dash_merged[dash_merged['tab_num'] == '17345']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,<NA>,<NA>,17345,NaT,NaT,NaN
1,2020-09-02,2,<NA>,<NA>,17345,NaT,NaT,NaN
2,2020-09-03,3,<NA>,<NA>,17345,NaT,NaT,NaN
3,2020-09-04,4,<NA>,<NA>,17345,NaT,NaT,NaN
4,2020-09-05,5,<NA>,<NA>,17345,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [39]:
tab_num_index = 2
dash_merged.loc[
    (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6) &
    (dash_merged['weekday'] == 0),
    'to_be_at_office'
] = 0

In [40]:
tab_num_index = 2
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] < finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] > start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([1, 2, 3, 4]))
    ),
    'to_be_at_office'
] = 1

In [41]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([0, 1, 4]))
    ),
    'to_be_at_office'
] = 1

In [42]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([2, 3]))
    ),
    'to_be_at_office'
] = 0

In [43]:
dash_merged[dash_merged['tab_num'] == df.tab_num.to_list()[1]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,16234,2020-09-20,2020-10-30,NaN
1,2020-09-02,2,0,0,16234,2020-09-20,2020-10-30,NaN
2,2020-09-03,3,0,0,16234,2020-09-20,2020-10-30,NaN
3,2020-09-04,4,0,0,16234,2020-09-20,2020-10-30,NaN
4,2020-09-05,5,0,0,16234,2020-09-20,2020-10-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,16234,2020-09-20,2020-10-30,NaN
118,2020-12-28,0,17,8,16234,2020-09-20,2020-10-30,NaN
119,2020-12-29,1,17,8,16234,2020-09-20,2020-10-30,NaN
120,2020-12-30,2,17,8,16234,2020-09-20,2020-10-30,NaN


In [44]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [45]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [46]:
dash_merged[dash_merged['tab_num'] == df.tab_num.to_list()[4]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,18456,2020-09-02,2020-09-30,NaN
1,2020-09-02,2,0,0,18456,2020-09-02,2020-09-30,0.0
2,2020-09-03,3,0,0,18456,2020-09-02,2020-09-30,0.0
3,2020-09-04,4,0,0,18456,2020-09-02,2020-09-30,0.0
4,2020-09-05,5,0,0,18456,2020-09-02,2020-09-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,18456,2020-09-02,2020-09-30,NaN
118,2020-12-28,0,17,8,18456,2020-09-02,2020-09-30,NaN
119,2020-12-29,1,17,8,18456,2020-09-02,2020-09-30,NaN
120,2020-12-30,2,17,8,18456,2020-09-02,2020-09-30,NaN


In [47]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [48]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [49]:
dash_merged[dash_merged['tab_num'] == df.tab_num.to_list()[5]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,19567,2020-09-02,2020-12-31,NaN
1,2020-09-02,2,0,0,19567,2020-09-02,2020-12-31,0.0
2,2020-09-03,3,0,0,19567,2020-09-02,2020-12-31,0.0
3,2020-09-04,4,0,0,19567,2020-09-02,2020-12-31,0.0
4,2020-09-05,5,0,0,19567,2020-09-02,2020-12-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,19567,2020-09-02,2020-12-31,NaN
118,2020-12-28,0,17,8,19567,2020-09-02,2020-12-31,0.0
119,2020-12-29,1,17,8,19567,2020-09-02,2020-12-31,0.0
120,2020-12-30,2,17,8,19567,2020-09-02,2020-12-31,0.0


In [50]:
# TO-DO поменять формат даты

In [51]:
dash_merged[['tab_num', 'ymd_date', 'to_be_at_office']].sort_values(by=['tab_num', 'ymd_date']).reset_index(drop=True)

,tab_num,ymd_date,to_be_at_office
0,15123,2020-09-01,NaN
1,15123,2020-09-02,1.0
2,15123,2020-09-03,1.0
3,15123,2020-09-04,1.0
4,15123,2020-09-05,NaN
...,...,...,...
605,19567,2020-12-27,NaN
606,19567,2020-12-28,0.0
607,19567,2020-12-29,0.0
608,19567,2020-12-30,0.0
